### Extract text from Pdf documents and apply NER

In [1]:
import pdfplumber, os
import os.path as osp
import pandas as pd
from pathlib import Path
from pdf_extract.services import file
from pdf_extract.config import global_config as glob
from pdf_extract.config import config
from pdf_extract.utils import utils
from importlib import reload
import fasttext

reload(glob)
reload(config)
reload(utils)

Environment Variable: UC_CODE_DIR has been set to default: /home/alexv84/Documents/GitHub/pdf_extract/src
Environment Variable: UC_DATA_DIR has been set to default: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data
Environment Variable: UC_LANG_ID has been set to default: /home/alexv84/Documents/Arbeit/Allianz/AZVers/fasttext_langdetect
Environment Variable: UC_DATA_PKG_DIR has been set to default: 
Environment Variable: UC_DB_CONNECTION has been set to default: postgresql://postgres...
Environment Variable: UC_PORT has been set to default: 5000
Environment Variable: UC_APP_CONNECTION has been set to default: 127.0.0.1


<module 'pdf_extract.utils.utils' from '/home/alexv84/Documents/GitHub/pdf_extract/src/pdf_extract/utils/utils.py'>

In [2]:
list_of_NEG_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'negatives'))
n_neg = len(list_of_NEG_docs)

list_of_POS_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'positives'))
n_pos = len(list_of_POS_docs)

print(n_pos)
print(n_neg)


12
13


In [3]:
# fname = 'CV4'

# i, page_objects, text = 0, {}, []
# # The open method returns an instance of the pdfplumber.PDF class.
# with pdfplumber.open(osp.join(glob.UC_DATA_DIR, f"example_cvs/{fname}.pdf")) as pdf:
#     while i < len(pdf.pages):
#         page = pdf.pages[i]
#         page_objects[str(i+1)] = page.extract_text(x_tolerance=1).split('\n')
#         #mystring = page_objects[str(i+1)].replace('\r', '').replace('\n', '')
#         text += page_objects[str(i+1)]
#         print(f"Page {i}")
#         #print(page.extract_text())
#         i += 1


In [4]:
#page_objects['1']

Read in all Pdfs in directory:

Postives only:

In [5]:
raw_corpus1 = pd.DataFrame(columns=['text', 'label'])

for z, fname in enumerate(list_of_POS_docs):
    #print(fname)
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"positives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus1.loc[z] = text  
raw_corpus1['label'] = 'positive'

Negatives only:

In [6]:
raw_corpus2 = pd.DataFrame(columns=['text','label'])

for z, fname in enumerate(list_of_NEG_docs):
    #print(fname)
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"negatives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus2.loc[z] = text   
raw_corpus2['label'] = 'negative'

In [7]:
raw_corpus = pd.concat([raw_corpus1,raw_corpus2], axis=0, ignore_index=True)
#raw_corpus

In [8]:
#page_objects

#page.extract_words()

In [9]:
#page_instance = page_objects['1']

In [10]:
#pdf.metadata
#page_instance

Preprocess text corpus:

In [11]:
from pdf_extract.resources import preprocessor as preproc

reload(preproc)

# Preprocess corpus:
cleaner = preproc.clean_text(language='english', lemma = False, stem = False)

X = raw_corpus['text']

X_cl = cleaner.fit_transform(X)

#docs = X_cl.tolist()    

Using english language.
Using 179 stop words.
Added 0 stopword(s).
Adding custom stop words...
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Finished preprocessing!


[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Prepare supervised FastText data set(s):

In [13]:
combine_fct = lambda x: '__label__{}'.format(x.label)

raw_corpus['text_cleaned'] = X_cl
raw_corpus['fasttext_label'] = raw_corpus.apply(combine_fct, axis = 1)

X = raw_corpus[['fasttext_label', 'text_cleaned']].reset_index(drop=True)   # keep label in X for txt-file below, will not be used directly in ft
#X.head()

In [14]:
from sklearn.model_selection import train_test_split

reload(utils)

X_train, X_valid, X_test, y_train, y_valid, y_test = utils.train_test_split_extend(X, X['fasttext_label'], test_size=[0.2, 0.2], random_state=42, shuffle=True)
#X_train, X_test, y_train, y_test = utils.train_test_split_extend(X, X['fasttext_label'], test_size=[0.2], random_state=42, shuffle=True)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)


(15, 2)
(5, 2)
(5, 2)


Prepare preprocessed data for fasttext training:

In [14]:
reload(file)

config_output = config.io['output']
Path(glob.UC_DATA_DIR).mkdir(parents=True, exist_ok=True)

# Train:
file.TXTService(verbose=True, root_path=glob.UC_DATA_DIR, **config_output['service']['TXTService1']).doWrite(X_train)

# Test:
file.TXTService(verbose=True, root_path=glob.UC_DATA_DIR, **config_output['service']['TXTService2']).doWrite(X_test)

# Dev:
file.TXTService(verbose=True, root_path=glob.UC_DATA_DIR, **config_output['service']['TXTService3']).doWrite(X_valid)

TXT Service output to file: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/train_data_fasttext.txt
TXT Service output to file: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/test_data_fasttext.txt
TXT Service output to file: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/dev_data_fasttext.txt


In [22]:
# Train model
#-------------

# Training word embeddings: 
#model = fasttext.train_supervised(input = os.path.join(glob.UC_DATA_DIR,'train_data_fasttext.txt'), wordNgrams = 2, dim=100, epoch=30, lr=1.0, verbose=2)

# Autotune hyperparameter:
model = fasttext.train_supervised(input=os.path.join(glob.UC_DATA_DIR,'train_data_fasttext.txt'), autotuneValidationFile=os.path.join(glob.UC_DATA_DIR,'dev_data_fasttext.txt'), dim=100, autotuneMetric="f1:__label__positive", autotuneDuration=300, verbose=2)


Progress: 100.0% Trials:  343 Best score:  1.000000 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  2225
Number of labels: 2


Done!


Progress: 100.0% words/sec/thread:    8130 lr:  0.000000 avg.loss:  0.694452 ETA:   0h 0m 0s


In [23]:
model.test(os.path.join(glob.UC_DATA_DIR,'dev_data_fasttext.txt'))

(5, 1.0, 1.0)

In [24]:
model.predict("I am a cook with 10 years of cooking experience. My hobbies are sports and cycling.")

(('__label__negative',), array([0.50001007]))

In [25]:
model.predict("I am a statistician from the university of Erlangen-Nuremberg")

(('__label__negative',), array([0.50001013]))

Get predictions for train/test set:

In [26]:
def predict(row):
    yhat, score = model.predict(row['text_cleaned'])
    return yhat[0]

y_train_pred = X_train.apply(predict,axis=1)
y_test_pred = X_test.apply(predict,axis=1)

In [27]:
from sklearn.metrics import classification_report

print('Train:')
print(classification_report(y_train, y_train_pred, target_names=['__label__negative', '__label__positive']))
print('Test:')
print(classification_report(y_test, y_test_pred, target_names=['__label__negative', '__label__positive']))

Train:
                   precision    recall  f1-score   support

__label__negative       0.80      1.00      0.89         8
__label__positive       1.00      0.71      0.83         7

         accuracy                           0.87        15
        macro avg       0.90      0.86      0.86        15
     weighted avg       0.89      0.87      0.86        15

Test:
                   precision    recall  f1-score   support

__label__negative       1.00      0.67      0.80         3
__label__positive       0.67      1.00      0.80         2

         accuracy                           0.80         5
        macro avg       0.83      0.83      0.80         5
     weighted avg       0.87      0.80      0.80         5



In [28]:
#page_instance.pdf.to_dict()

In [20]:
#page_instance.extract_words()

### NER:

In [37]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_lg")

Use pretrained language identification:

In [60]:
import fasttext

model = fasttext.load_model(osp.join(glob.UC_LANG_ID, 'lid.176.bin'))

label, score = model.predict("Hallo ich studiere Mathe in München", k=2)  # top 2 matching languages
label[0]


'__label__de'

In [38]:
i = 1
#raw_text = X.loc[i]
raw_text = X_cl.loc[i]
#raw_text

In [37]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."

In [39]:
text1 = NER(raw_text)

In [40]:
# for word in text1.ents:
#     print(word.text,word.label_)

In [41]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

In [42]:
spacy.explain("GPE")

'Countries, cities, states'

In [44]:
#displacy.render(text1,style="ent",jupyter=True)

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [28]:
URL="https://www.zeebiz.com/markets/currency/news-cryptocurrency-news-today-june-12-bitcoin-dogecoin-shiba-inu-and-other-top-coins-prices-and-all-latest-updates-158490"

In [29]:
html_content = requests.get(URL).text

In [30]:
soup = BeautifulSoup(html_content, "lxml")

In [31]:
body=soup.body.text

In [32]:
body= body.replace('\n', ' ')
#body= body.replace('t', ' ')
#body= body.replace('r', ' ')
#body= body.replace('xa0', ' ')
#body=re.sub(r'[^ws]', '', body)

In [148]:
#body

In [149]:
text3= NER(body)